In [1]:
import cv2
import numpy as np
import dlib
import math
from math import hypot
from scipy.spatial import distance
from datetime import datetime
from datetime import timedelta
import datetime
import time
from collections import Counter

#from imutils import face_utils

In [6]:
#캠 설정 :(0), 노트북 자체
cap = cv2.VideoCapture(0)

#얼굴 설정
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

#폰트 설정
font = cv2.FONT_HERSHEY_PLAIN

#Counters
head_right_frame = 0 #있음
head_right_count_per_s = 0 #있음

head_left_frame = 0 # 있음
head_left_count_per_s = 0 # 있음

blinking_frame = 0  #있음

#시간 따라 계산하기 위한 변수
blinking_time_list = [] #있음

# the direction of one's gaze
gaze_direction = ""

#종합조건 변수
#머리 회전
# the direction of one's face
head_direction =""
head_direction_frame = 0
head_direction_total_frame = []

#졸음 검출
sleep_false_exit_cnt = 0
sleep_ing = False

### 눈동자 검출
gaze_direction_list = []
gaze_direction_point = False

#시간 측정기
tm = cv2.TickMeter()

# 유저 ID 및 시간 기록
user_dict = {}
user_count = 0
current_users = 0
current_faces = []

#눈의 수평, 수직선 긋기 위한 함수
def midpoint(p1, p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

#양쪽 눈감김 함수로 정의
def get_blinking_ratio(eye_points,facial_landmarks) :
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    
    #수평선
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    #수평선/수직선(시선 추적에 방해 될 수 있으므로 주석처리)
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0),2)
    #ver_line = cv2.line(frame, center_top, center_bottom,(0,255,0),2)

    #수평선 길이
    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    #수직선 길이
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    #눈이 감기는 비율
    ratio = (hor_line_lenght/ver_line_lenght)
    return ratio

def get_gaze_ratio(eye_points, facial_landmarks):
    
    
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
        
    
    #얼굴 전체를 흑백으로 바꿔줄 마스크(원래 프레임이랑 같아야한다.)        
    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8) # 이 영역에 눈을 그릴것
    
    #흑백 얼굴 영역에 눈 그리기
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
    
    #눈의 영역을 설정하는 코드
    #cv2.polylines(frame, [left_eye_region], True, (0, 0, 255), 2)
    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])
    
    gray_eye = eye[min_y: max_y, min_x: max_x]
    #gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    
    if gray_eye.size == 0 or gray_eye.shape[0] < 1 or gray_eye.shape[1] < 1:
    # 올바르지 않은 영역이 감지되면, 오류 메시지를 출력하거나 기본값을 반환할 수 있습니다.
        print("Invalid eye region detected.")
        return 0
    
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    if threshold_eye is None:
        # thresholding에 실패한 경우, 오류 메시지를 출력하거나 기본값을 반환할 수 있습니다.
        print("Thresholding failed.")
        return 0
    
    height_t, width_t = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height_t, 0: int(width_t/2)]
    left_side_white = cv2.countNonZero(left_side_threshold)
    
    right_side_threshold = threshold_eye[0: height_t, int(width_t/2):width_t]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else: 
        gaze_ratio = left_side_white/right_side_white

    return gaze_ratio


while True:
    _, frame = cap.read()
    #start_time = datetime.datetime.now()
    #print(start_time.second)
    
    #프레임 회색조 변경
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    if len(faces) > current_users:
        # 새로운 얼굴이 감지된 경우
        for _ in range(current_users, len(faces)):
            user_count += 1
            user_id = f"user-{user_count}"
            user_dict[user_id] = {'start_time': datetime.datetime.now(), 'end_time': None}
        
        current_users = len(faces)  # 현재 사용자 수 업데이트
        

    elif len(faces) < current_users:
        # 어떤 얼굴이 사라진 경우
        users_to_update = [user for user, data in user_dict.items() if data['end_time'] is None]
        for user_id in users_to_update[-(current_users - len(faces)):]:  # 끝 시간이 아직 기록되지 않은 사용자 업데이트
            user_dict[user_id]['end_time'] = datetime.datetime.now()

        current_users = len(faces)
    
    for i, face in enumerate(faces):
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        num_faces = len(faces)
        landmarks = predictor(gray, face)

        face_id = f"user-{user_count+1}"
        #current_faces.add(face_id)
        
        
        ### 눈감김 비율 계산
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2

        ### Gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47], landmarks)
        
        #왼쪽, 오른쪽 눈의 임계값 평균
        gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye)/2

        ### 머리 회전 반경 계산
        end_nose_point = (landmarks.part(29).x, landmarks.part(29).y)
        left_libs_point = (landmarks.part(4).x, landmarks.part(4).y)
        right_libs_point = (landmarks.part(12).x, landmarks.part(12).y)

        #코와 입의 길이 계산
        nose_line_len_left = hypot(left_libs_point[0]-end_nose_point[0],left_libs_point[1]-end_nose_point[1])
        nose_line_len_right = hypot(right_libs_point[0]-end_nose_point[0],right_libs_point[1]-end_nose_point[1])
        nose_ratio = nose_line_len_left/nose_line_len_right

        # 사용자 정보 및 검출된 얼굴 수 표시 (왼쪽 상단에 위치)
        if num_faces > 0:
            user_info = 'user 1: {} faces'.format(num_faces)
            cv2.putText(frame, user_info, (10, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)  # 왼쪽 상단에 텍스트 배치
            
            
        #눈감는 비율 감지 및 명령
        if blinking_ratio >= 4.8 :            
            blinking_frame += 1
            #print(blinking_frame) #이상없음
            if blinking_frame == 10:
                blinking_frame_to_s += 1
                blinking_frame = 0
                #print(blinking_frame_to_s) #이상없음
                if blinking_frame_to_s == 3:
                    blinking_time_list.append(datetime.datetime.now())
                    blinking_frame_to_s = 0                
                    if len(blinking_time_list) >= 3 :
                        blinking_cnt_per_s = blinking_time_list[(len(blinking_time_list))-1] - blinking_time_list[(len(blinking_time_list))-3]
                        #print("초", blinking_cnt_per_s.seconds)
                        if blinking_cnt_per_s.seconds <= 60:
                            #print("조건절 전", sleep_ing) 이상 없음
                            sleep_ing = True
                            #print("조건절 후", sleep_ing) 조건 확인 후 True로 출력 됨
            #print("elif 전:sleep_ing", sleep_ing)  조건 변경 되는거 확인 무한 루프를 탄다.                 
        elif sleep_ing == True:        
            #print("elif문아래:", sleep_ing)
            sleep_false_exit_cnt += 1
            #print(sleep_false_exit_cnt)
            if sleep_false_exit_cnt >= 500:                
                sleep_false_exit_cnt = 0
                #print("false전",sleep_ing)
                sleep_ing = False
                #print("false후",sleep_ing)
        else:
            #print("els아래", sleep_ing)
            blinking_frame = 0
            blinking_frame_to_s = 0
            
        if sleep_ing == False :
            pop_text = "awake"
            
        else:
            pop_text = "falling asleep"
        cv2.putText(frame, pop_text,  (10, 90), font, 1, (255, 0, 0), 2, cv2.LINE_AA)     
           
        #Gaze detection 여기 아래서 부터 시작
        #동체 추적 비율
        if gaze_ratio <= 1:
            gaze_direction = "RIGHT"
            gaze_direction_list.append(2)            
            #cv2.putText(frame, "RIGHT", (50,150), font, 2, (0,0,255), 3)            
            
        elif 1 < gaze_ratio < 1.5:
            gaze_direction = "CENTER"
            gaze_direction_list.append(1)
            #cv2.putText(frame, "CENTER", (50,150), font, 2, (0,0,255), 3)
        else :
            gaze_direction = "LEFT"
            gaze_direction_list.append(0)
            #cv2.putText(frame, "LEFT", (50,150), font, 2, (0,0,255), 3)
        
        if len(gaze_direction_list) >= 151:
            gaze_direction_list.pop(0)
            #print(len(gaze_direction_list)) #150출력
            if (Counter(gaze_direction_list)[0] == 150) or (Counter(gaze_direction_list)[2] == 150):
                #print("true전", gaze_direction_point)
                gaze_direction_point = True
                #print("true후",gaze_direction_point)
                if (Counter(gaze_direction_list)[0] <= 100) or (Counter(gaze_direction_list)[2] <= 100):
                    gaze_direction_point = False
                    #print("조건후 False확인:",gaze_direction_point)
                    
        
        #얼굴 회전 반경에 의한 좌우 구분    
        if nose_ratio <= 0.6 :
            head_direction = "RIGHT"
            head_right_frame += 1
            #print("head_right_frame:", head_right_frame)            
            if head_right_frame == 10:                
                head_right_count_per_s += 1
                #print("head_right_count_per_s:", head_right_count_per_s)
                head_right_frame = 0
                #print("head_right_frame:", head_right_frame)                
                if head_right_count_per_s % 20 == 0:
                    #print("오른쪽을 본지 20초 지남")             
                    head_right_count_per_s = 0
                    #print("head_right_count_per_s:",head_right_count_per_s)
                    
            #cv2.putText(frame, "RIGHT", (300,150), font, 2, (0,0,255), 3)
            
        elif nose_ratio >= 1.5:
            head_direction = "LEFT"
            head_left_frame += 1
            #print("head_left_frame:", head_left_frame)
            
            if head_left_frame == 10:                
                head_left_count_per_s += 1
                #print("head_left_count_per_s:", head_left_count_per_s)
                head_left_frame = 0
                #print("head_left_frame:", head_left_frame)
                
                if head_left_count_per_s % 20 == 0:
                    #print("왼쪽을 본지 20초 지남")
                    head_left_count_per_s = 0
                    #print("head_left_count_per_s:",head_left_count_per_s)                    
            #cv2.putText(frame, "LEFT", (300,150), font, 2, (0,0,255), 3)
            
        else :
            head_right_frame = 0 
            head_right_count_per_s = 0
            head_left_frame = 0
            head_left_count_per_s = 0
            head_direction = "CENTER"
            #cv2.putText(frame, "CENTER", (300,150), font, 2, (0,0,255), 3)
            
        #gaze_ration 여기까지 복사함
        #화면에 입계값 출력하기(gaze_ratio)
        #cv2.putText(frame, str(gaze_ratio), (50,100), font, 2, (0,0,255), 3)
        
        #threshold_eye = cv2.resize(threshold_eye, None, fx=5, fy=5)
        #eye = cv2.resize(gray_eye, None, fx=5, fy=5)
        #cv2.imshow("Eye", eye)
        #cv2.imshow("Threshold", threshold_eye)
        #cv2.imshow("Left eye", left_eye)
        #cv2.imshow("left", left_side_threshold)
        #cv2.imshow("right", right_side_threshold)
        
        #얼굴 감지 유무 확인
        #얼굴 감지 후 시선 방향으로 집중 정도
        if faces[0] != 0:            
            face_on_time = datetime.datetime.now()
            head_direction_frame += 1
            
            if head_direction == "RIGHT" :
                head_direction_total_frame.append(2)
            elif head_direction == "CENTER":
                head_direction_total_frame.append(1)
            else :
                head_direction_total_frame.append(0)
                
            if len(head_direction_total_frame) >= 600 :                
                head_direction_total_frame.pop(0)
                #print(gaze_direction_point)
                if ((len(head_direction_total_frame) - (Counter(head_direction_total_frame)[1])) <= 150) and (sleep_ing == False) and (gaze_direction_point == False):
                    cv2.putText(frame, "Under the Concentrate", (10, 70), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
                else :
                    cv2.putText(frame, "Under the Disperse", (10, 70), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
            elif (sleep_ing == True):
                cv2.putText(frame, "falling asleep", (10, 90), font, 1, (255, 0, 0), 2, cv2.LINE_AA)           
            else:
                cv2.putText(frame, "Under the Disperse", (10, 70), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
                
            cv2.putText(frame, "Detection has begun", (10, 30), font, 2, (255, 0, 0), 2, cv2.LINE_AA)
      
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(5)
    if key == 27:
        break
        
# 모든 사용자의 감지 시간을 출력합니다.
for user_id, times in user_dict.items():
    print(f"{user_id}: Start: {times['start_time']}, End: {times['end_time']}")
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
.py로 갔을 때

In [7]:
# 일반적으로 웹캠 불러오기
cam = cv2.VideoCapture(0)
ret, frame = cam.read()

# 기존 방식으로 연결이 안될 경우
# 여기서 숫자 0은 웹캠의 채널 인덱스

cam = cv2.VideoCapture(cv2.CAP_DSHOW+0)
ret, frame = cam.read()